In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [2]:
from google.colab import drive

In [3]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
batch_size = 32
num_classes = 4  # Replace with the number of classes in your dataset
input_shape = (224, 224, 3)  # Change dimensions as per your dataset images

In [5]:
train_data_dir = '/content/drive/MyDrive/dataset/train'
test_data_dir = '/content/drive/MyDrive/dataset/test'

In [6]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)


In [7]:
test_datagen = ImageDataGenerator(rescale=1.0/255)


In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 1951 images belonging to 4 classes.


In [9]:
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 106 images belonging to 4 classes.


In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)


94765736/94765736 [==============================] - 5s 0us/step


In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)


In [12]:
predictions = Dense(num_classes, activation='softmax')(x)

In [13]:
model = Model(inputs=base_model.input, outputs=predictions)

In [14]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [16]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 20
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    validation_data=test_generator,
    validation_steps=len(test_generator),
    epochs=epochs
)


Epoch 1/20
61/61 [==============================] - ETA: 0s - loss: 1.3207 - accuracy: 0.4126 

In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test accuracy:", test_accuracy)

In [ ]:
model.save('cotton_disease_detection_model.h5')

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

In [ ]:
epochs_range = range(1, epochs + 1)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()

In [ ]:
plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_accuracy, label='Training Accuracy')
plt.plot(epochs_range, val_accuracy, label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

In [ ]:
plt.tight_layout()
plt.show()
plt.savefig('AccVal_acc')